In [11]:
# In the prompt, it is given that the reserve price is unknown but
# grows linearly from 900 to 1000.

# I assume that the pdf of the reserve price is parametrized by a (0 < a< 1)
# and it is given by
# f(x) = a/100 + (1-a)*(x-900)/5000

# Let us denote the low bid as x1+900 and the high bid as x2+900
# Doing the integral, the expected reward is given by,

def reward_fun(a,x1,x2):
    result = 0
    result += a*(100-x1)*x1/100.0 +a*(100-x2)*(x2-x1)/100.0
    result += (1-a)*(100-x1)*(x1**2)/10000.0 +(1-a)*(100-x2)*(x2**2-x1**2)/10000.0
    return result

# Brute force the optimal solutions
def find_optimal(a):
    optim_pairs = 0
    optim_rewards = 0
    for x1 in range(0,101):
        for x2 in range(x1+1, 101):
            curr_reward = reward_fun(a, x1,x2)
            if curr_reward > optim_rewards:
                optim_rewards = curr_reward
                optim_pairs = (x1,x2)
    return optim_rewards, optim_pairs

In [12]:
a = 0

reward, bids  = find_optimal(float(a))
print("reward : ", reward)
print("(low bid, high bid) : ", bids)

reward :  20.4152
(low bid, high bid) :  (52, 78)


In [13]:
import statistics as stat

d = {}

for a in range(0, 1000):
    reward, bids  = find_optimal(a / 1000)
    d[a] = (reward, bids)

low_bids = [v[1][0] for v in d.values()]
high_bids = [v[1][1] for v in d.values()]
rewards = [v[0] for v in d.values()]

print('low:', stat.fmean(low_bids, weights=rewards) + 900)
print('high:', stat.fmean(high_bids, weights=rewards) + 900)

low: 940.7735076246277
high: 971.9804966330775
